In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
import tensorflow.keras.layers as Layer

BASE_PATH = '../input/g2net-gravitational-wave-detection/'

In [ ]:
!pip install -q nnAudio

In [ ]:
def npy_header_offset(npy_path):
    with open(str(npy_path), 'rb') as f:
        if f.read(6) != b'\x93NUMPY':
            raise ValueError('Invalid NPY file.')
        version_major, version_minor = f.read(2)
        if version_major == 1:
            header_len_size = 2
        elif version_major == 2:
            header_len_size = 4
        else:
            raise ValueError('Unknown NPY file version {}.{}.'.format(version_major, version_minor))
        header_len = sum(b << (8 * i) for i, b in enumerate(f.read(header_len_size)))
        header = f.read(header_len)
        if not header.endswith(b'\n'):
            raise ValueError('Invalid NPY file.')
        return f.tell()

def apply_raw_path(row, is_train=True): 
    file_name = row[0]
    if is_train:
        return os.path.join(
            BASE_PATH, 'train',
            file_name[0],
            file_name[1],
            file_name[2],
            file_name + ".npy")
    else:
        return os.path.join(
            BASE_PATH, 'test',
            file_name[0],
            file_name[1],
            file_name[2],
            file_name + ".npy")

In [ ]:
import torch
from nnAudio.Spectrogram import CQT1992v2

transform=CQT1992v2(sr=2048, fmin=22, fmax=512, hop_length=64)

def cqt_2_rgb(data): # in order to use efficientnet we need 3 dimension images
    data = data.numpy()
    
    wave0 = np.concatenate((data[0], data[0]))
    wave1 = np.concatenate((data[1], data[1]))
    wave2 = np.concatenate((data[2], data[2]))
    
    wave0 = transform(torch.from_numpy(wave0))
    wave1 = transform(torch.from_numpy(wave1))
    wave2 = transform(torch.from_numpy(wave2))
    image = tf.convert_to_tensor([np.array(wave0)[0], np.array(wave1)[0], np.array(wave2)[0]])
    image = tf.transpose(image,(1,2,0))
    return image

def wave_transform(data):
    data = tf.reshape(tf.io.decode_raw(data, tf.float64), (3,4096))
    data = tf.cast(data, tf.float32)

    data0 = data[0]/data[0][tf.argmax(data[0])]
    data1 = data[1]/data[1][tf.argmax(data[1])]
    data2 = data[2]/data[2][tf.argmax(data[2])]
    data = tf.stack([data0, data1, data2])
    
    data = tf.py_function(
            cqt_2_rgb,
            [data],
            [tf.float32])
    
    data = tf.convert_to_tensor(data[0])
    data = tf.math.minimum(data, .99)

    return data

In [ ]:
# df = pd.read_csv(os.path.join(BASE_PATH, 'training_labels.csv'))
sample_submission = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')

In [ ]:
from tensorflow.keras.applications.efficientnet import *
from tensorflow.keras.applications.efficientnet import preprocess_input

model = tf.keras.models.load_model('/kaggle/input/full-b5-dec95-4e4/full_b5_dec95_4e4')


In [ ]:
# df['file_path'] = df.apply(apply_raw_path, args=(True,), axis=1)
# df['target'] = df['target'].astype('float32')

In [ ]:
BATCH_SIZE = 64

df_test = sample_submission
df_test['file_path'] = df_test.apply(apply_raw_path, args=(False,), axis=1)

ds_test = tf.data.FixedLengthRecordDataset(
    df_test['file_path'],
    98304,
    header_bytes=128,
    num_parallel_reads=4).map(wave_transform)


In [ ]:
# test_data = ds_test.take(5000).batch(BATCH_SIZE).prefetch(buffer_size=16)
# preds = model.predict(test_data)
# submission = pd.DataFrame({'id':sample_submission['id'][5000],'target':preds.reshape(-1)})

In [ ]:
test_data = ds_test.batch(BATCH_SIZE).prefetch(buffer_size=16)
preds = model.predict(test_data)
submission = pd.DataFrame({'id':sample_submission['id'],'target':preds.reshape(-1)})

In [ ]:
submission.to_csv('submission.csv',index=False)

In [ ]:
submission[0:20]